<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto </font></h1>

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Import and Explore Dataset</a>

2. <a href="#item2">Explore Neighborhoods in Toronto</a>

3. <a href="#item3">Analyze Each Neighborhood</a>

4. <a href="#item4">Cluster Neighborhoods</a>

5. <a href="#item5">Examine Clusters</a>    
</font>
</div>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


## 1. Import and Explore Dataset

In [2]:
# Read csv with previously processed data
df = pd.read_csv('extracted_df_toronto_coordinates.csv')

In [3]:
df.head()

,Unnamed: 0,Unnamed: 0.1,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,0,M3A,North York,Parkwoods,43.752420,-79.329242
1,1,1,M4A,North York,Victoria Village,43.730600,-79.313265
2,2,2,M5A,Downtown Toronto,Harbourfront,43.650295,-79.359166
3,3,3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.723270,-79.451286
4,4,4,M7A,Queen's Park,Queen's Park,43.661150,-79.391715


In [4]:
df1 = df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

#### Use geopy library to get the latitude and longitude values of Toronto City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>trnt_explorer</em>, as shown below.

In [5]:
address = 'Toronto City, ON'

geolocator = Nominatim(user_agent="trnt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [6]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df1['Latitude'], df1['Longitude'], df1['Borough'], df1['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [7]:
East_Toronto = df1[df1['Borough'] == 'East Toronto']
Central_Toronto = df1[df1['Borough'] == 'Central Toronto']
West_Toronto = df1[df1['Borough'] == 'West Toronto']

In [8]:
toronto_df = [East_Toronto,Central_Toronto,West_Toronto]
toronto = pd.concat(toronto_df, axis = 0).reset_index(drop=True)

In [9]:
toronto.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676531,-79.295425
1,M4K,East Toronto,"The Danforth West ,Riverdale",43.683178,-79.355105
2,M4L,East Toronto,"The Beaches West ,India Bazaar",43.667965,-79.314667
3,M4M,East Toronto,Studio District,43.660629,-79.334855
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 East...,43.648690,-79.385440
5,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133
6,M5N,Central Toronto,Roselawn,43.711941,-79.419120
7,M4P,Central Toronto,Davisville North,43.712755,-79.388514
8,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.694785,-79.414405
9,M4R,Central Toronto,North Toronto West,43.714523,-79.406960


In [17]:
toronto.shape

(20, 5)

Let's get the geographical coordinates of East Toronto, Central Toronto and West Toronto

In [12]:
# East Toronto, Central Toronto and West Toronto, we use a loop
address = ['East Toronto, ON', 'Central Toronto, ON', 'West Toronto, ON']
latitude = []
longitude = []

for i in range(len(address)):
    geolocator = Nominatim(user_agent="trnt_explorer")
    location = geolocator.geocode(address[i])
    latitude.append(location.latitude)
    longitude.append(location.longitude)
    print('The geograpical coordinate of ' +  address[i] + ' are  {}, {}.'.format(latitude[i], longitude[i]))

The geograpical coordinate of East Toronto, ON are  43.626243, -79.396962.
The geograpical coordinate of Central Toronto, ON are  43.653963, -79.387207.
The geograpical coordinate of West Toronto, ON are  43.653963, -79.387207.


In [13]:
latitude[1]

43.653963

Let's visualize all Toronto named neighborhoods.

In [15]:
# create map of Manhattan using latitude and longitude values
map_toronto_central = folium.Map(location=[latitude[1], longitude[1]], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_central)  
    
map_toronto_central

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [18]:
CLIENT_ID = 'FLZED3BEIVB5HLS3ZMJTNWBSFL3WL0QM4T1SVXW0YI3UU5A4' # your Foursquare ID
CLIENT_SECRET = 'W231AZOJUDH3JZGC4XKVZX215KE33DINLTCVSWZT2D3F11KN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FLZED3BEIVB5HLS3ZMJTNWBSFL3WL0QM4T1SVXW0YI3UU5A4
CLIENT_SECRET:W231AZOJUDH3JZGC4XKVZX215KE33DINLTCVSWZT2D3F11KN


#### Let's explore the first neighborhood in our dataframe.

Get the neighborhood's name.

In [19]:
toronto.loc[0, 'Neighbourhood']

'The Beaches'

Get the neighborhood's latitude and longitude values.

In [20]:
neighbourhood_latitude = toronto.loc[0, 'Latitude'] # neighbourhood latitude value
neighbourhood_longitude = toronto.loc[0, 'Longitude'] # neighbourhood longitude value

neighbourhood_name = toronto.loc[0, 'Neighbourhood'] # neighbourhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of The Beaches are 43.67653121600006, -79.29542499999997.


#### Now, let's get the top 100 venues that are in The Beaches within a radius of 500 meters.

First, let's create the GET request URL. Name your URL **url**. We will use the **explore** function from the API Foursquare

In [21]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=FLZED3BEIVB5HLS3ZMJTNWBSFL3WL0QM4T1SVXW0YI3UU5A4&client_secret=W231AZOJUDH3JZGC4XKVZX215KE33DINLTCVSWZT2D3F11KN&v=20180605&ll=43.67653121600006,-79.29542499999997&radius=500&limit=100'

Send the GET request and examine the results

In [22]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5df3e7155fb726001be063b5'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.68103122050006,
    'lng': -79.28921469650147},
   'sw': {'lat': 43.672031211500055, 'lng': -79.30163530349846}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

From the Foursquare lab in the previous module, we know that all the information is in the *items* key. Before we proceed, let's borrow the **get_category_type** function from the Foursquare lab.

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Now we are ready to clean the json and structure it into a *pandas* dataframe.

In [24]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

In [25]:
nearby_venues

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.postalCode,venue.venuePage.id
0,e-0-4bd461bc77b29c74a07d9282-0,0,"[{'summary': 'This spot is popular', 'type': '...",4bd461bc77b29c74a07d9282,Glen Manor Ravine,Glen Manor,Queen St.,43.676821,-79.293942,"[{'label': 'display', 'lat': 43.67682094413784...",123,CA,Toronto,ON,Canada,"[Glen Manor (Queen St.), Toronto ON, Canada]","[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",0,[],NaN,NaN
1,e-0-4ad4c062f964a52011f820e3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4ad4c062f964a52011f820e3,The Big Carrot Natural Food Market,125 Southwood Dr,NaN,43.678879,-79.297734,"[{'label': 'display', 'lat': 43.678879, 'lng':...",320,CA,Toronto,ON,Canada,"[125 Southwood Dr, Toronto ON M4E 0B8, Canada]","[{'id': '50aa9e744b90af0d42d5de0e', 'name': 'H...",0,[],M4E 0B8,75150878
2,e-0-4b8daea1f964a520480833e3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4b8daea1f964a520480833e3,Grover Pub and Grub,676 Kingston Rd.,at Main St.,43.679181,-79.297215,"[{'label': 'display', 'lat': 43.67918143494101...",328,CA,Toronto,ON,Canada,"[676 Kingston Rd. (at Main St.), Toronto ON M4...","[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",0,[],M4E 1R4,NaN
3,e-0-56afcad6498e05333bf42031-3,0,"[{'summary': 'This spot is popular', 'type': '...",56afcad6498e05333bf42031,Glen Stewart Ravine,NaN,NaN,43.676300,-79.294784,"[{'label': 'display', 'lat': 43.67629984029563...",57,CA,NaN,NaN,Canada,[Canada],"[{'id': '4bf58dd8d48988d162941735', 'name': 'O...",0,[],NaN,NaN
4,e-0-4df91c4bae60f95f82229ad5-4,0,"[{'summary': 'This spot is popular', 'type': '...",4df91c4bae60f95f82229ad5,Upper Beaches,NaN,NaN,43.680563,-79.292869,"[{'label': 'display', 'lat': 43.68056321147582...",493,CA,Toronto,ON,Canada,"[Toronto ON, Canada]","[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",0,[],NaN,NaN


In [26]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Glen Stewart Ravine,Other Great Outdoors,43.676300,-79.294784
4,Upper Beaches,Neighborhood,43.680563,-79.292869


And how many venues were returned by Foursquare?

In [27]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


## 2. Explore Neighbourhoods around Toronto Central

#### Let's create a function to repeat the same process to all the neighborhoods in East Toronto, Central Toronto and West Toronto

In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighbourhood and create a new dataframe called *around_toronto_central_venues*.

In [35]:
around_toronto_central_venues = getNearbyVenues(names=toronto['Neighbourhood'],
                                   latitudes=toronto['Latitude'],
                                   longitudes=toronto['Longitude'])


The Beaches
The Danforth West ,Riverdale
The Beaches West ,India Bazaar
Studio District 
Business Reply Mail Processing Centre 969 Eastern 
Lawrence Park
Roselawn 
Davisville North 
Forest Hill North,Forest Hill West 
North Toronto West 
The Annex,North Midtown ,Yorkville
Davisville 
Moore Park,Summerhill East 
Deer Park,Forest Hill SE ,Rathnelly,South Hill,Summerhill West 
Dovercourt Village,Dufferin 
Little Portugal,Trinity
Brockton ,Exhibition Place,Parkdale Village
High Park,The Junction South 
Parkdale,Roncesvalles
Runnymede,Swansea


#### Let's check the size of the resulting dataframe

In [36]:
print(around_toronto_central_venues.shape)
around_toronto_central_venues.head()

(491, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676531,-79.295425,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676531,-79.295425,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676531,-79.295425,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676531,-79.295425,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676531,-79.295425,Upper Beaches,43.680563,-79.292869,Neighborhood


Let's check how many venues were returned for each neighbourhood

In [37]:
around_toronto_central_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Brockton ,Exhibition Place,Parkdale Village",65,65,65,65,65,65
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
Davisville,26,26,26,26,26,26
Davisville North,7,7,7,7,7,7
"Deer Park,Forest Hill SE ,Rathnelly,South Hill,Summerhill West",6,6,6,6,6,6
"Dovercourt Village,Dufferin",17,17,17,17,17,17
"Forest Hill North,Forest Hill West",2,2,2,2,2,2
"High Park,The Junction South",2,2,2,2,2,2
Lawrence Park,3,3,3,3,3,3


#### Let's find out how many unique categories can be curated from all the returned venues

In [38]:
print('There are {} uniques categories.'.format(len(around_toronto_central_venues['Venue Category'].unique())))

There are 144 uniques categories.


## 3. Analyze Each Neighborhood

In [40]:
# one hot encoding
toronto_onehot = pd.get_dummies(around_toronto_central_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = around_toronto_central_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Travel,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Music Store,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Other Great Outdoors,Park,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Skating Rink,Smoke Shop,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.

In [41]:
toronto_onehot.shape

(491, 145)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [42]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Board Shop,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Dessert Shop,Diner,Discount Store,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,French Restaurant,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Travel,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Mac & Cheese Joint,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Music Store,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Other Great Outdoors,Park,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Poutine Place,Pub,Ramen Restaurant,Record Shop,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shoe Store,Skating Rink,Smoke Shop,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Steakhouse,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,"Brockton ,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.015385,0.015385,0.000000,0.000000,0.046154,0.000000,0.030769,0.015385,0.000000,0.000000,0.000000,0.000000,0.015385,0.000000,0.000000,0.000000,0.000000,0.000000,0.061538,0.015385,0.000000,0.015385,0.092308,0.00,0.015385,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.0,0.0000,0.000000,0.015385,0.015385,0.015385,0.000000,0.000000,0.000000,0.000000,0.000000,0.015385,0.015385,0.000000,0.00,0.000000,0.00000,0.000000,0.061538,0.00,0.000000,0.000000,0.00,0.015385,0.000000,0.000000,0.000000,0.030769,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.0,0.030769,0.0,0.015385,0.000000,0.030769,0.015385,0.00,0.000000,0.015385,0.000000,0.000000,0.000000,0.000000,0.015385,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.015385,0.015385,0.00,0.00,0.000000,0.0,0.000000,0.015385,0.000000,0.00000,0.000000,0.000000,0.00,0.015385,0.000000,0.00,0.000000,0.061538,0.00,0.000000,0.046154,0.015385,0.015385,0.000000,0.000000,0.00000,0.015385,0.000000,0.015385,0.00,0.000000,0.030769,0.000000,0.000000,0.00,0.015385,0.015385,0.000000,0.015385,0.000000,0.0,0.030769,0.015385,0.000000,0.000000,0.000000
1,Business Reply Mail Processing Centre 969 East...,0.020000,0.000000,0.000000,0.010000,0.010000,0.020000,0.000000,0.010000,0.000000,0.040000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.040000,0.000000,0.010000,0.000000,0.100000,0.01,0.000000,0.020000,0.000000,0.010000,0.000000,0.000000,0.000000,0.00000,0.010000,0.000000,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.00000,0.000000,0.010000,0.00,0.020000,0.000000,0.01,0.000000,0.000000,0.010000,0.000000,0.020000,0.010000,0.00,0.0,0.0,0.000000,0.000000,0.0,0.040000,0.0,0.000000,0.01000

#### Let's confirm the new size

In [43]:
toronto_grouped.shape

(20, 145)

#### Let's print each neighborhood along with the top 5 most common venues

In [45]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brockton ,Exhibition Place,Parkdale Village----
                    venue  freq
0             Coffee Shop  0.09
1  Furniture / Home Store  0.06
2              Restaurant  0.06
3                    Café  0.06
4          Sandwich Place  0.05


----Business Reply Mail Processing Centre 969 Eastern ----
         venue  freq
0  Coffee Shop  0.10
1   Steakhouse  0.04
2         Café  0.04
3          Bar  0.04
4        Hotel  0.04


----Davisville ----
                venue  freq
0        Dessert Shop  0.12
1  Italian Restaurant  0.08
2                Café  0.08
3         Pizza Place  0.08
4      Sandwich Place  0.08


----Davisville North ----
               venue  freq
0              Hotel  0.29
1     Clothing Store  0.14
2               Park  0.14
3     Breakfast Spot  0.14
4  Food & Drink Shop  0.14


----Deer Park,Forest Hill SE ,Rathnelly,South Hill,Summerhill West ----
                 venue  freq
0          Coffee Shop  0.33
1   Light Rail Station  0.33
2          Supermarket  0.17

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [47]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Brockton ,Exhibition Place,Parkdale Village",Coffee Shop,Furniture / Home Store,Restaurant,Café,Sandwich Place,Bakery,Bar,Gym,Italian Restaurant,Supermarket
1,Business Reply Mail Processing Centre 969 East...,Coffee Shop,Hotel,Café,Bar,Steakhouse,Sushi Restaurant,Pub,Restaurant,Pizza Place,Thai Restaurant
2,Davisville,Dessert Shop,Italian Restaurant,Pizza Place,Coffee Shop,Café,Sandwich Place,American Restaurant,Seafood Restaurant,Indian Restaurant,Fast Food Restaurant
3,Davisville North,Hotel,Park,Gym,Breakfast Spot,Clothing Store,Food & Drink Shop,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
4,"Deer Park,Forest Hill SE ,Rathnelly,South Hill...",Coffee Shop,Light Rail Station,Liquor Store,Supermarket,Food Court,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [48]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 3, 2, 4, 0], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [49]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676531,-79.295425,0,Health Food Store,Other Great Outdoors,Trail,Pub,Neighborhood,Falafel Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
1,M4K,East Toronto,"The Danforth West ,Riverdale",43.683178,-79.355105,0,Park,Grocery Store,Restaurant,Bus Line,Discount Store,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Food & Drink Shop
2,M4L,East Toronto,"The Beaches West ,India Bazaar",43.667965,-79.314667,0,Park,Sandwich Place,Pet Store,Fruit & Vegetable Store,Gym,Movie Theater,Pizza Place,Pub,Fast Food Restaurant,Fish & Chips Shop
3,M4M,East Toronto,Studio District,43.660629,-79.334855,0,Diner,Café,Italian Restaurant,American Restaurant,Bar,Sandwich Place,Pizza Place,Brewery,Sushi Restaurant,Coffee Shop
4,M7Y,East Toronto,Business Reply Mail Processing Centre 969 East...,43.648690,-79.385440,0,Coffee Shop,Hotel,Café,Bar,Steakhouse,Sushi Restaurant,Pub,Restaurant,Pizza Place,Thai Restaurant


Finally, let's visualize the resulting clusters

In [58]:
# create map
map_clusters = folium.Map(location=[latitude[2], longitude[2]], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'],toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. 

#### Cluster 1

In [52]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Other Great Outdoors,Trail,Pub,Neighborhood,Falafel Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
1,East Toronto,0,Park,Grocery Store,Restaurant,Bus Line,Discount Store,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Food & Drink Shop
2,East Toronto,0,Park,Sandwich Place,Pet Store,Fruit & Vegetable Store,Gym,Movie Theater,Pizza Place,Pub,Fast Food Restaurant,Fish & Chips Shop
3,East Toronto,0,Diner,Café,Italian Restaurant,American Restaurant,Bar,Sandwich Place,Pizza Place,Brewery,Sushi Restaurant,Coffee Shop
4,East Toronto,0,Coffee Shop,Hotel,Café,Bar,Steakhouse,Sushi Restaurant,Pub,Restaurant,Pizza Place,Thai Restaurant
7,Central Toronto,0,Hotel,Park,Gym,Breakfast Spot,Clothing Store,Food & Drink Shop,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
9,Central Toronto,0,Park,Gym Pool,Playground,Garden,Yoga Studio,Falafel Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
10,Central Toronto,0,Sandwich Place,Café,History Museum,Italian Restaurant,Pub,Pizza Place,Pharmacy,Modern European Restaurant,Mexican Restaurant,Liquor Store
11,Central Toronto,0,Dessert Shop,Italian Restaurant,Pizza Place,Coffee Shop,Café,Sandwich Place,American Restaurant,Seafood Restaurant,Indian Restaurant,Fast Food Restaurant
12,Central Toronto,0,Restaurant,Playground,Gym,Convenience Store,Yoga Studio,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


#### Cluster 2

In [54]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Central Toronto,1,Health & Beauty Service,IT Services,Yoga Studio,Farmers Market,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant


#### Cluster 3

In [55]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,West Toronto,2,Park,Convenience Store,Yoga Studio,Farmers Market,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant


#### Cluster 4

In [56]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,3,Home Service,Park,Yoga Studio,Farmers Market,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant


#### Cluster 5

In [57]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Central Toronto,4,Bus Line,Gym / Fitness Center,Swim School,Farmers Market,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant
